# NRMS : Neural News Recommendation with Multi-Head Self-Attention
Content-based neural 뉴스 추천   
뉴스 인코더는 multi-head self-attention을 이용해 단어들 사이의 상호작용을 모델링함으로써 뉴스 제목으로부터 뉴스를 표현하는 방법을 학습한다. 유저 인코더는 learn representations of users from their browsed news and use multihead self-attention to capture the relatedness between the news. 또한, 추가적인 어텐션을 적용하여 to learn more informative news and user representations by selecting important words and news.

In [1]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMSModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]
Tensorflow version: 2.7.3


In [2]:
epochs = 5
seed = 42
batch_size = 32
MIND_type = 'demo'


tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'nrms.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)


hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs,
                          show_step=10)

100%|█████████████████████████████████████| 17.0k/17.0k [00:10<00:00, 1.57kKB/s]
100%|█████████████████████████████████████| 9.84k/9.84k [00:05<00:00, 1.71kKB/s]
100%|█████████████████████████████████████| 95.0k/95.0k [00:30<00:00, 3.16kKB/s]


In [3]:
iterator = MINDIterator
model = NRMSModel(hparams, iterator, seed=seed)
print(model.run_eval(valid_news_file, valid_behaviors_file))

2022-07-12 13:15:42.170547: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-12 13:15:42.238950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 13:15:42.259847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 13:15:42.260146: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

{'group_auc': 0.4792, 'mean_mrr': 0.2059, 'ndcg@5': 0.2045, 'ndcg@10': 0.2701}


In [4]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

step 1080 , total_loss: 1.5135, data_loss: 1.3306: : 1086it [04:16,  4.23it/s]
586it [00:00, 721.66it/s]
236it [00:13, 17.59it/s]
7538it [00:00, 9550.49it/s] 


at epoch 1
train info: logloss loss:1.513202964381399
eval info: group_auc:0.5759, mean_mrr:0.2437, ndcg@10:0.3293, ndcg@5:0.2583
at epoch 1 , train time: 256.6 eval time: 18.3


step 1080 , total_loss: 1.4181, data_loss: 1.3042: : 1086it [04:08,  4.37it/s]
586it [00:00, 725.25it/s]
236it [00:13, 17.88it/s]
7538it [00:00, 14322.72it/s]


at epoch 2
train info: logloss loss:1.4183067778197442
eval info: group_auc:0.5988, mean_mrr:0.256, ndcg@10:0.3457, ndcg@5:0.2721
at epoch 2 , train time: 248.6 eval time: 17.8


step 1080 , total_loss: 1.3771, data_loss: 1.2525: : 1086it [04:15,  4.25it/s]
586it [00:00, 716.03it/s]
236it [00:13, 17.71it/s]
7538it [00:00, 15577.07it/s]


at epoch 3
train info: logloss loss:1.3768836972243659
eval info: group_auc:0.6085, mean_mrr:0.2647, ndcg@10:0.3569, ndcg@5:0.2852
at epoch 3 , train time: 255.4 eval time: 17.9


step 1080 , total_loss: 1.3473, data_loss: 1.3042: : 1086it [04:07,  4.39it/s]
586it [00:00, 713.12it/s]
236it [00:13, 17.75it/s]
7538it [00:00, 14477.06it/s]


at epoch 4
train info: logloss loss:1.3474418966590151
eval info: group_auc:0.6129, mean_mrr:0.2704, ndcg@10:0.3624, ndcg@5:0.292
at epoch 4 , train time: 247.5 eval time: 17.9


step 1080 , total_loss: 1.3284, data_loss: 1.3910: : 1086it [04:07,  4.39it/s]
586it [00:00, 715.88it/s]
236it [00:13, 17.54it/s]
7538it [00:00, 14298.11it/s]


at epoch 5
train info: logloss loss:1.3284732866243323
eval info: group_auc:0.6122, mean_mrr:0.2677, ndcg@10:0.3609, ndcg@5:0.2882
at epoch 5 , train time: 247.3 eval time: 18.1
CPU times: user 11min 4s, sys: 36.4 s, total: 11min 40s
Wall time: 22min 25s


In [5]:
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

586it [00:00, 677.62it/s]
236it [00:13, 17.79it/s]
7538it [00:00, 15162.45it/s]


{'group_auc': 0.6122, 'mean_mrr': 0.2677, 'ndcg@5': 0.2882, 'ndcg@10': 0.3609}


In [6]:
sb.glue('res_syn', res_syn)

In [7]:
model_path = os.path.join(data_path, 'model')
os.makedirs(model_path, exist_ok=True)
model.model.save_weights(os.path.join(model_path, 'nrms_ckpt'))

In [8]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()


586it [00:00, 664.88it/s]
236it [00:13, 17.86it/s]
7538it [00:00, 14020.33it/s]
7538it [00:00, 63168.14it/s]
